In [7]:
from typing import List

# Heap
Python documentation for [heapq](https://docs.python.org/3/library/heapq.html).

#### Min or Max Heap?
Python `heapq` uses minHeap. 
For maxHeap, invert comparisons using
- negate values (eg. uses `-distance` instead of `distance`)
- wrapper class with `__lt__()` method

Tuple comparisons: break tie with subsequent entries.

 


### K Closest Points to Origin

Use a maxHeap of size `K` to hold the K minimum-distance points. The farther points are upper in the heap and gets popped off. The K remaining points in heap are the K closest points.

When heap reaches `K` elements: decide whether or not to **replace `root` with the new element then sift down** 

https://leetcode.com/problems/k-closest-points-to-origin/

### Top K Frequent Words (uses lexicographical order to break tie)

Use a minHeap of size `K` to hold the K most frequent words. The heap's root is the K-th most frequent word (the least frequent one among the K-th most frequent words). 

When heap reaches `K` words: 
- If the new word is less frequent than the `root`  --> new word is less frequent than all the K words in heap --> ignore the new word.
- Else if the new word is more frequent tha `root` --> replace `root` with the new word, then siftdown. (call `heapq.heapreplace(arr, newWord)`

Use a wrapper class and implement `__lt__()` for more comprehensive comparsion.

https://leetcode.com/problems/top-k-frequent-words/

### The Skyline Problem
https://leetcode.com/problems/the-skyline-problem/

Process the walls of each building.
(only the walls matter since every skyline point's x-coordinate is 
determined by some wall's x-coordinate)
        
    wall: (x, -height)
    Push wall's height into a maxheap (so negative height).
    -> Sorted by x-coordinates.  
    -> If same x-coordinates, the 'starting wall comes first
    -> If both are starting walls, the taller one comes first
    -> If both are ending walls, the shorter one comes first
    => Just use -height for starting walls. Use height for ending walls.

    Kepp track of `maxHeight` so far (initilaized as 0)

    Push "starting wall"'s height into maxHeap of heights
        - If the max height updates:
            1. Skyline point found.
        - Else:
            1. Not a skyline point.
            2. Keep its height in the heap.

    Remove "ending wall"'s height from the maxHeap
        - If the max height updates (shrinks)
            1. x <-- get the x coordinate of this ending wall
            2. h <-- the new max height left.
            3. Record (x,h) as a skyline point
        - Else:
            1. Not a skyline point


In [3]:
## Heap O(n^2)
def getSkyline(buildings):

    walls = []  # each wall is (x, -height)
    for left, right, height in buildings:
        walls.append((left, -height))
        walls.append((right, height))

    walls.sort()    # n log n

    skyline = []

    maxHeight = 0
    heap = [0]   # max heap of the walls' heights
    for x, height in walls:  #O(n)
        ## Starting wall
        if height < 0:
            # Push its height. (Want: MaxHeap. Use: -height)
            heapq.heappush(heap, height)
            # If this updates the maxHeight, record skyline point
            if maxHeight != heap[0]:    # taller building found
                skyline.append([x, -height])
                maxHeight = heap[0]
            # Else, the start of this building is covered by another
            # building that already started.

        ## Ending wall
        elif height > 0:
            # Remove its height from the maxheap
            heap.remove(-height)
            heapq.heapify(heap)
            # If this changes the maxHeight, then we're done with some 
            # previous taller building, and move on to the next shorter one
            if maxHeight != heap[0]:
                skyline.append([x, -heap[0]])
            maxHeight = heap[0]

    return skyline
    """
    walls = [*(2,start,-10)*, *(3,start,-15)*, *(5,start,12)*, *
    *(7,end,-15)*, *(9,end,-10)*, *(12,end,12)]

    maxHeight = -12
    heightsHeap = [0]

    skyline points: [2,10], [3,15], [7,12], [12,0]


    Time: O(n^2)
    Space: O(n)
    """    

In [ ]:
## Divide and Conquer (N log N)
def getSkylineDivideAndConquer(buildings):
    
    

# Stack & Monotonic Stack

Better use the doubly linked list implementation: 
`stack = collections.deque()`

pop() | append(x) | peek stack[-1]


### Next Greater Element
https://leetcode.com/problems/next-greater-element-i/

Similar: [Buildings with Oceanview](https://leetcode.com/problems/buildings-with-an-ocean-view/)

### Daily Temperature
Indices matter


### Trapping Rain Water
Given `n` non-negative integers representing an elevation map where the width of each bar is `1`, compute how much water it can trap after raining.



### 接雨水
Also monotonic stack? 

### Brick Wall
- Cumulative array
- count the votes for each "crossable position" among all rows.
- Hashtable find max value.

### Find celebrity
Def. celebrity: everybody knows him, but he knows nobody.
(---knows---> is a 1-way arrow)

calling `knows(A,B)` determines that only one of them could be the celebrity.

## HashMap

### Contains Almost Duplicate III

- Sliding window of size t+1
- Bucket width `w = t+1`
- Divide each element by `w` => The bucket id 
- Place each element `num` in `bucket[bucket_id]`
- Check for an element's almost duplicate by looking at buckets at `[bucket_id]`, `[bucket_id-1]` and check if the numbers differ within `k`, and `[bucket_id+1]` and check if the numbers differ within `k`.

(see Wk7 -> bucket sort for more)

In [8]:
def containsNearbyAlmostDuplicate(nums: List[int], k: int, t: int) -> bool:
    '''
    Buckets of width t+1
    [+] Elements in the same bucket must be "almost duplicates"
    [!] Possible to have "almost duplicate" in nearby buckets 
    (bucket(x-1), bucket(x+1))

    t = 3
    bucket_id:  num // (t+1) 
    *      
                *
    0,1,2,3   4,5,6,7  8,9,10,11
    bucket0   bucket1  bucket2

    Sliding window of size k+1

    k = 2
    1 5 9 1 5 9
          ^  
remove  ^
    0 1 2 3 4
    '''

    buckets = {}    # {id: number from nums}


    for i, num in enumerate(nums):
        bucket_id = num // (t+1) 

        # Check if num is already in this bucket, or the left or right bucket. 
        if bucket_id in buckets:
            return True
        if (bucket_id - 1 in buckets) and abs(num - buckets[bucket_id-1]) <= t:
            return True
        if (bucket_id + 1 in buckets) and abs(num - buckets[bucket_id+1]) <= t:
            return True

        # Add num to the correct bucket
        buckets[bucket_id] = num

        # Remove the window's leftmost element from buckets 
        # before the whole window shifts rightward
        if i-k >= 0:
            to_remove = nums[i-k]
            bucket_id_to_remove = to_remove // (t+1)
            buckets.pop(bucket_id_to_remove)

    return False

    '''
    Time: O(N)  where N is the length of nums.
    Space: O(N)
    '''